### Imports and CUDA

In [142]:
# Matplotlib
import requests
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Torch
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [143]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Objective

#### To develop a model that predicts taxi availability within a specific area for the next three hours. This means that if the model is run at 12 PM, it will provide predicted taxi availability for 1 PM, 2 PM, and 3 PM.

The area of interest is defined by the following geographical boundaries:

    North: 1.35106
    South: 1.32206
    East: 103.97839
    West: 103.92805

To identify the taxis currently available within this region, we use the TaxiAvailabilityScript.py.

This script collects real-time data, which serves as input for our predictive model.

By leveraging historical taxi availability trends and real-time data, our model aims to provide accurate forecasts, helping commuters, ride-hailing services, and urban planners make informed decisions.


# **To-Do List for Taxi Availability Prediction**

## **Step 1: Cleaning the Taxi Availability Data**
The first step involves retrieving and preprocessing the taxi availability dataset. The dataset consists of the following columns:

1. **DateTime**  
2. **Taxi Available Throughout Singapore**  
3. **Taxi Available in Selected Box Area**  
4. **Coordinates[]**  

For our specific use case, **the coordinates column will not be used for now**.  

To prepare the data for the neural network:  
- **Inputs:** We will use `DateTime` and `Taxi Available Throughout Singapore` as features.  
- **Output:** `Taxi Available in Selected Box Area` will be the target variable.  
- **DateTime Conversion:** Since `DateTime` is not in a format suitable for neural networks, we will extract relevant features:
  - **IsWeekend**: A binary feature (1 if it's a weekend, 0 otherwise).  
  - **Hour**: Transformed into a numerical value between **1 and 24** (avoiding 0, which may cause training issues).  

---

## **Step 2: Adding Additional Features**  
*(Partially completed; will be refined over time)*  

Aside from the existing columns, we aim to incorporate additional features that may improve prediction accuracy:  

1. **ERP Rates (Electronic Road Pricing) at the given time and location**  
   - Uncertain if this will significantly impact predictions. Further analysis is needed.  

2. **Number of LTA (Land Transport Authority) gantry locations**  
   - Again, its usefulness remains uncertain—further evaluation required.  

3. **Traffic Incidents in the Selected Area**  
   - A script (`TrafficIncidentScript.py`) has been written to update `traffic_incident.csv` with the latest traffic incidents.  
   - Over time, as the dataset grows, we hope this feature will become useful.  

4. **Number of Taxi Stands in the Area**  
   - Currently **not useful** because our area of interest is fixed.  
   - However, if we allow dynamic selection of areas in the future, this could become relevant.  

5. **Temperature at a Given Time and Date** *(To be implemented)*  

6. **Rainfall Data** *(To be implemented)*  

To ensure all features align properly, we will **synchronize all datasets based on DateTime** before feeding them into the model.  

---

## **Step 3: Creating the Training-Test Split**  
- Initially, we will perform an **80/20 Training-Test split** for simplicity.  
- In the future, we may introduce a **Training-Validation-Test split** to further refine model performance.  

---

## **Step 4: Building the Model**  
We will begin with an **LSTM model**, as LSTMs are well-suited for time-series forecasting.  
- **Initial Limitation:** The model, in its basic form, will only predict the next hour.  
- **Future Improvement:** A **sliding window approach** will be explored and implemented to extend predictions further.  

---

## **Step 5: Model Evaluation and Improvement**  
- After the initial model is trained, we will assess its performance.  
- Based on results, we will explore potential improvements, such as hyperparameter tuning, architectural modifications, or additional feature engineering.  

---

This structured approach will guide the development of a robust and accurate taxi availability prediction model. 🚖💡


## **Preparing the taxi_availability data here.**

Normalization of certain inputs are done as well, but I am unsure if it is the right thing to do as well.

In [144]:
taxi_availability_file_path = "taxi_availability.csv"

taxi_df = pd.read_csv(taxi_availability_file_path, delimiter=",")
taxi_df_coordinates = taxi_df["Coordinates[]"]
taxt_df_datetime = taxi_df["DateTime"]
taxi_df = taxi_df.drop(columns = "Coordinates[]")
taxi_df["DateTime"] = pd.to_datetime(taxi_df["DateTime"])

taxi_df["IsWeekend"] = (taxi_df["DateTime"].dt.weekday >= 5).astype(int)
taxi_df["Hour"] = taxi_df["DateTime"].dt.hour + 1  # Convert 0-23 to 1-24
taxi_df = taxi_df.drop(columns = "DateTime")
# Save this just in case we need it in the future.

In [145]:
print(taxi_df.dtypes)
# taxi_df=taxi_df[:5120]
numeric_columns = taxi_df.select_dtypes(include=['int64', 'int32','float64','object']).columns
print("numeric_columns",numeric_columns)
taxi_df[numeric_columns] = taxi_df[numeric_columns].astype('float32')
numeric_columns = taxi_df.select_dtypes(include=['int64', 'int32','float64','object']).columns

# Convert selected columns to float32
taxi_df[numeric_columns] = taxi_df[numeric_columns].astype('float32')




Taxi Available throughout SG           int64
Taxi Available in Selected Box Area    int64
IsWeekend                              int64
Hour                                   int32
dtype: object
numeric_columns Index(['Taxi Available throughout SG', 'Taxi Available in Selected Box Area',
       'IsWeekend', 'Hour'],
      dtype='object')


In [146]:
#---------------Normalise-----------------------
# Drop 'DateTime' as it's no longer needed
# Normalize the 'Hour' and 'IsWeekend' columns (if needed)


data_min = taxi_df.min(axis=0)
data_max = taxi_df.max(axis=0)
taxi_df_normalized = (taxi_df - data_min) / (data_max - data_min)

# scaler = MinMaxScaler()
# taxi_df[["Hour", "IsWeekend", "Taxi Available throughout SG","Taxi Available in Selected Box Area"]] = scaler.fit_transform(taxi_df[["Hour", "IsWeekend","Taxi Available throughout SG","Taxi Available in Selected Box Area"]])
# taxi_df = taxi_df.apply(pd.to_numeric, errors='coerce')
taxi_df_output_normalized  = taxi_df_normalized["Taxi Available in Selected Box Area"]
taxi_df_normalized = taxi_df_normalized.drop(columns = "Taxi Available in Selected Box Area")
taxi_df_normalized.to_csv("checker.csv", index=False)  # Set index=False to exclude row numbers

# Convert to NumPy arrays
input_data = taxi_df_normalized.values  # Shape: (5120, num_features)
output_data = taxi_df_output_normalized.values  # Shape: (5120,)
# taxi_df_output = taxi_df["Taxi Available in Selected Box Area"]
# input_data = taxi_df.values  # Shape: (5120, num_features)
# output_data = taxi_df_output.values  # Shape: (5120,)

print("Input Data: ",input_data.shape)
print("Output Data: ",output_data.shape)

# Define sequence length

seq_length = 24

# Function to create continuous sequences
#Checked, it works correctly
def create_sequences(data, labels, seq_length):
    xs, ys = [], []
    for i in range(0, len(data), seq_length):  # Start from 0 and increment by seq_length
        if i + seq_length <= len(data):  # Make sure there are enough data points for the sequence
            xs.append(data[i:i + seq_length])  # Input sequence (continuous)
            ys.append(labels[i + seq_length - 1])  # Corresponding target (last value of the sequence)
    return np.array(xs), np.array(ys)


Input Data:  (8689, 3)
Output Data:  (8689,)


In [147]:

X, y = create_sequences(input_data, output_data, seq_length)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)  # Add channel dim
y = torch.tensor(y[:, None], dtype=torch.float32)

# Split into training (80%) and testing (20%)
train_size = int(0.8 * len(X))
print("train_size", train_size)
test_size = len(X) - train_size

trainX, testX = X[:train_size], X[train_size:]
trainY, testY = y[:train_size], y[train_size:]

print("trainX: ", trainX[169])
# Convert to PyTorch tensors
trainX = torch.tensor(trainX, dtype=torch.float32)  # Shape: (train_size, 24, num_features)
trainY = torch.tensor(trainY, dtype=torch.float32)  # Shape: (train_size,)
testX = torch.tensor(testX, dtype=torch.float32)    # Shape: (test_size, 24, num_features)
testY = torch.tensor(testY, dtype=torch.float32)    # Shape: (test_size,)

train_dataset = TensorDataset(trainX, trainY)
test_dataset = TensorDataset(testX, testY)

batch_size = 17  # You can adjust the batch size as needed
#Shuffle is false because you need it to be in time sequenced, but please double check
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example of accessing a batch of data
for inputs, targets in train_loader:
    print(f'Inputs: {inputs.shape}, Targets: {targets.shape}')
    break  # Only print the first batch for verification

train_size 289
trainX:  tensor([[0.6706, 0.0000, 1.0000],
        [0.6774, 0.0000, 0.9565],
        [0.6750, 0.0000, 0.9130],
        [0.6546, 0.0000, 0.8696],
        [0.7002, 0.0000, 0.8261],
        [0.5988, 0.0000, 0.7826],
        [0.5899, 0.0000, 0.7391],
        [0.6233, 0.0000, 0.6957],
        [0.6552, 0.0000, 0.6522],
        [0.6703, 0.0000, 0.6087],
        [0.5754, 0.0000, 0.5652],
        [0.6750, 0.0000, 0.5217],
        [0.6177, 0.0000, 0.4783],
        [0.7191, 0.0000, 0.4348],
        [0.6464, 0.0000, 0.3913],
        [0.5399, 0.0000, 0.3478],
        [0.5745, 0.0000, 0.3043],
        [0.6461, 0.0000, 0.2609],
        [0.3800, 0.0000, 0.2174],
        [0.5115, 0.0000, 0.1739],
        [0.5118, 0.0000, 0.1304],
        [0.5423, 0.0000, 0.0870],
        [0.6162, 0.0000, 0.0435],
        [0.6387, 0.0000, 0.0000]])
Inputs: torch.Size([17, 24, 3]), Targets: torch.Size([17, 1])


C:\Users\nicho\AppData\Local\Temp\ipykernel_26116\1113359302.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainX = torch.tensor(trainX, dtype=torch.float32)  # Shape: (train_size, 24, num_features)
C:\Users\nicho\AppData\Local\Temp\ipykernel_26116\1113359302.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainY = torch.tensor(trainY, dtype=torch.float32)  # Shape: (train_size,)
C:\Users\nicho\AppData\Local\Temp\ipykernel_26116\1113359302.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testX = torch.tensor(testX, dtype=t

In [148]:
print("test_loader",len(test_loader))

test_loader 5


In [149]:
class LSTM_pt(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTM_pt, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, output_dim),
            torch.nn.ReLU()  # Helps prevent constant outputs
        )
    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.randn(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.randn(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out, hn, cn

In [150]:
def train(model, dataloader,num_layers, hidden_size, num_epochs, learning_rate, device, ):
    # Set the loss function and optimizer
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()  # Set the model to training mode
    
    for epoch in range(num_epochs):
        loss=0
        hidden_state, cell_state = None, None   
        optimizer.zero_grad() #Double check if this is the right position
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            if batch_idx == len(train_loader) - 1:  
                break  # Skip the last batch
            # Initialize hidden state and cell state for each batch
            # Forward pass
            output, cell_state, hidden_state = model(inputs,cell_state, hidden_state)
            # Calculate loss
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            hidden_state = hidden_state.detach()
            cell_state = cell_state.detach()
        if epoch %100 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss/len(dataloader)}')
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss/len(dataloader)}')

In [ ]:
# Define the model parameters
# Following the research paper's instructions
input_size = 3
hidden_size = 50
num_layers = 1 # Can be changed to stack multiple LSTM layers!
output_size = 1
dataloader = train_loader
#Create the model
model = LSTM_pt(input_size, hidden_size, num_layers, output_size).to(device)
train(model, dataloader,num_layers, hidden_size, num_epochs = 200, learning_rate = 0.01, device = device)


Epoch 1/200, Loss: 0.0039790174923837185
Epoch 101/200, Loss: 0.00022179012012202293
Epoch 200/200, Loss: 0.00036620264290831983


In [152]:
print(len(test_loader))

5


In [153]:

model.eval()

# Initialize variables to track loss
loss_value = 0
num_batches = 0

# Define the loss function
criterion = torch.nn.MSELoss()

# Initialize hidden state and cell state
hidden_state, cell_state = None, None  

# Disable gradient computation for validation
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        if batch_idx == len(test_loader) - 1:  
            break  # Skip the last batch
        # Forward pass
        output, cell_state, hidden_state = model(inputs, cell_state, hidden_state)


        # Compute loss
        # Compute loss
        loss_value += criterion(output, targets)
# Compute average loss
loss_value = loss_value / (len(test_loader) -1 )
print("Predicted output: ", output)
print("True Output: ", targets)
# Print validation results
print(f'Average Validation Loss: {loss_value:.4f}')


Predicted output:  tensor([[0.3822],
        [0.3988],
        [0.4560],
        [0.4205],
        [0.3392],
        [0.3564],
        [0.3761],
        [0.3996],
        [0.4812],
        [0.2361],
        [0.3644],
        [0.3851],
        [0.3779],
        [0.4250],
        [0.4227],
        [0.4455],
        [0.4243]])
True Output:  tensor([[0.3577],
        [0.3496],
        [0.2846],
        [0.4309],
        [0.4593]])
Average Validation Loss: 0.0061


In [154]:

# model.eval()

# # Initialize variables to track loss
# loss_value = 0
# num_batches = 0

# # Define the loss function
# criterion = torch.nn.MSELoss()

# # Initialize hidden state and cell state
# hidden_state, cell_state = None, None  

# # Disable gradient computation for validation
# with torch.no_grad():
#     for batch_idx, (inputs, targets) in enumerate(test_loader):
#         if batch_idx == len(test_loader) - 1:  
#             break  # Skip the last batch
#         # Forward pass
#         output, cell_state, hidden_state = model(inputs, cell_state, hidden_state)
#         print("Predicted output: ", output)
#         print("True Output: ", targets)

#         # Compute loss
#         # Compute loss
#         loss_value += criterion(output, targets)
# # Compute average loss
# loss_value = loss_value / (len(test_loader) -1 )

# # Print validation results
# print(f'Average Validation Loss: {loss_value:.4f}')
